# Import thư viện cần thiết

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

Đọc dữ liệu

In [2]:
train = pd.read_csv('../data/train_preprocessed.csv')
train.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,0.0,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [3]:
test = pd.read_csv('../data/test_preprocessed.csv')
test.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
0,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,1,...,4,0,16,9,0,1,2.25,0.25,272.25,16
1,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,1,...,41,256,1681,9,0,1,2.25,0.25,272.25,1681
2,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,1,...,41,289,1681,9,0,1,2.25,0.25,272.25,1681
3,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,0,...,59,256,3481,1,256,0,1.00,0.00,256.00,3481
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,18,121,324,1,0,1,0.25,64.00,121.00,324


In [4]:
ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat((train, test)).reset_index(drop=True)

# Xây dựng mô hình học máy

## Loại bỏ những thuộc tính dư thừa

### Tạo biến ordinal từ dữ liệu đã được one-hot encode

Các thuộc tính như `epared`, `etecho`, `eviv` và `instlevel` có thể được chuyển về dạng dữ liệu ordinal với quy ước **(bad, regular, good) -> (0, 1, 2)**

In [5]:
def get_numeric(data, status_name):
    status_cols = [s for s in data.columns.tolist() if status_name in s]
    status_df = data[status_cols]
    status_df.columns = list(range(status_df.shape[1]))
    status_numeric = status_df.idxmax(1)
    status_numeric.name = status_name
    data = pd.concat([data, status_numeric], axis=1)
    return data

In [6]:
status_name_list = ['epared', 'etecho', 'eviv', 'instlevel']
for status_name in status_name_list:
    all_data = get_numeric(all_data, status_name)

### Xóa những thuộc tính không cần thiết

Nhóm nhận thấy có những thuộc tính có thể được xác định bằng những thuộc tính khác trong dữ liệu.

- Nhóm thuộc tính sau có thể được tạo ra bằng sự kết hợp từ `r4h` và `r4m`:
    ```
    r4t1, persons younger than 12 years of age
    r4t2, persons 12 years of age and older
    r4t3, Total persons in the household
    ```

- Các thuộc tính sau mang cùng ý nghĩa với `hogar_total`:
    ```
    tamhog, size of the household
    tamviv, number of persons living in the household
    hhsize, household size
    r4t3, Total persons in the household
    ```

- `v18q` có thể được tạo ra từ `v18q1`.
- `mobilephone` có thể được tạo ra từ `qmobilephone`.
- `area2` có thể suy ra từ `area1`.
- `female` có thể suy ra từ `male`.
- `epared1~3`, `etecho1~3`, `eviv1~3`, `instlevel1~9` do đã được chuyển đổi thành dữ liệu ordinal nên sẽ không dùng đến nữa.

In [7]:
redundant_features = ['r4t1', 'r4t2', 'r4t3', 
                      'tamhog', 'tamviv', 'hhsize', 'r4t3', 
                      'v18q', 'mobilephone', 'area2', 'female',
                      'epared1', 'epared2', 'epared3', 
                      'etecho1', 'etecho2', 'etecho3',
                      'eviv1', 'eviv2', 'eviv3', 
                      'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9']
all_data.drop(columns=redundant_features, inplace=True)

## Trích lọc đặc trưng bằng thông số thống kê

Để kết hợp dữ liệu của từng cá nhân vào dữ liệu của cả hộ gia đình, ta cần tổng hợp dữ liệu đó cho từng hộ gia đình. Cách đơn giản nhất để thực hiện việc này là nhóm dữ liệu theo `idhogar` rồi tổng hợp dữ liệu. Tuy nhiên, các dữ liệu **boolean** có thể giống nhau, và sẽ tạo ra nhiều cột dư thừa mà sau đó chúng ta sẽ cần phải loại bỏ sau khi triển khai.

In [8]:
ind_bool = ['dis', 'male',
            'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
            'parentesco1', 'parentesco2',  'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 
            'parentesco7', 'parentesco8',  'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12', 'instlevel']

ind_ordered = ['escolari', 'age']

In [9]:
range = lambda x: x.max() - x.min()
range.__name__ = 'range_'
ind_agg = all_data.groupby('idhogar')[ind_ordered + ind_bool].agg(['mean', 'max', 'min', 'sum', 'count', 'std', range])

new_cols = []
for col in ind_agg.columns.levels[0]:
    for stat in ind_agg.columns.levels[1]:
        new_cols.append(f'{col}-{stat}')

ind_agg.columns = new_cols
ind_agg.head()

,escolari-mean,escolari-max,escolari-min,escolari-sum,escolari-count,escolari-std,escolari-range_,age-mean,age-max,age-min,...,parentesco12-count,parentesco12-std,parentesco12-range_,instlevel-mean,instlevel-max,instlevel-min,instlevel-sum,instlevel-count,instlevel-std,instlevel-range_
idhogar,,,,,,,,,,,,,,,,,,,,,
000a08204,8.666667,14,0,26,3,7.571878,14,20.666667,30,4,...,3,0.0,0,4.666667,7,0,14,3,4.041452,7
000bce7c4,2.500000,5,0,5,2,3.535534,5,61.500000,63,60,...,2,0.0,0,0.500000,1,0,1,2,0.707107,1
001845fb0,10.250000,14,6,41,4,3.304038,8,35.500000,52,19,...,4,0.0,0,4.000000,7,2,16,4,2.160247,5
001ff74ca,8.000000,16,0,16,2,11.313708,16,19.000000,38,0,...,2,0.0,0,3.500000,7,0,7,2,4.949747,7
003123ec2,3.250000,7,0,13,4,3.774917,7,12.750000,24,1,...,4,0.0,0,1.250000,3,0,5,4,1.500000,3


In [10]:
corr_matrix = ind_agg.corr()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any((abs(upper[column]) > 0.95) & (abs(upper[column]) == 1))]
print(f'There are {len(to_drop)} correlated columns to remove')

There are 33 correlated columns to remove


In [11]:
all_data = all_data.merge(ind_agg, on = 'idhogar', how = 'left')
all_data.drop(columns=ind_bool+ind_ordered+to_drop, inplace=True)
print('Number of features after dropping the individual level features', all_data.shape[1])

Number of features after dropping the individual level features 230


## Tạo thêm cột phụ biểu hiện thêm thông tin của hộ gia đình

Trong một hộ gia đình, cần biết rằng người trưởng thành có khả năng làm việc là trụ cột của gia đình, trẻ em và người lớn trên 65 tuổi thì không làm việc. Do đó việc quyết định việc xem xét đánh giá tình trạng của một gia đình không phụ thuộc nhiều vào tỉ lệ các nhóm đối tượng trong hộ gia đình. Vì vậy ta sẽ tạo thêm một số cột liên quan đến vấn đề này.

In [12]:
def extract_features(df):
    df['adult_num'] = df['hogar_adul'] - df['hogar_mayor'] # số lượng người trưởng thành còn khả năng làm việc
    df['head_is_adult'] = (df['adult_num'] > 0).astype(int) # có người trưởng thành là trụ cột
    df['adult_rate'] = df['adult_num'] / df['hogar_total'] # tỉ lệ người trưởng thành
    
    df['dependency_num'] = df['hogar_nin'] + df['hogar_mayor'] # số người phụ thuộc
    df['dependency_rate'] = df['dependency_num'] / df['hogar_total'] # tỉ lệ
    
    df['adult_dependency_rate'] = df['adult_num'] / (df['dependency_num'] + 0.1) # tỉ lệ người trưởng thành trên người phụ thuọc
    df['children_rate'] = df['hogar_nin'] / df['hogar_total'] # tỉ lệ trẻ em trong gia đình
    df['elder_rate'] = df['hogar_mayor'] / df['hogar_total'] # tỉ lệ người già trong gia đình

    df['rent_per_person'] = df['v2a1'] / df['hogar_total'] # giá thuê nhà tính trên mỗi người
    df['rent_per_adult'] = df['v2a1'] / (df['adult_num']+0.1) # gía thuê nhà tính trênngười thưởng thành

    df['bedroom_per_person'] = df['bedrooms'] / df['hogar_total'] # số lượng người trung bình mỗi phòng
    df['bedroom_per_adult'] = df['bedrooms'] / (df['adult_num'] + 0.1) # số lượng người trung bình mỗi phòng cho người trưởng thành
    
for df in [train, test]:
    extract_features(df)

## Xây dựng mô hình

### Chuẩn bị dữ liệu

In [13]:
train = all_data[:ntrain][:]
test = all_data[ntrain:][:]

Lấy dữ liệu dùng để huấn luyện model từ tập `train`.

In [14]:
X = train.drop(columns=['Id', 'idhogar', 'Target'])
y = train.Target.to_numpy().astype('int') - 1

Lấy dữ liệu dùng để huấn luyện từ tập `test`.

In [15]:
data_test_id = list(test.Id)
data_test = test.drop(columns=['Id', 'idhogar', 'Target'])

Chia dữ liệu huấn luyện thành tập train và test.

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Xây dựng mô hình

In [17]:
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [18]:
model_lgbm = lgb.LGBMClassifier(class_weight='balanced', boosting_type='dart',
                                drop_rate=0.9, min_data_in_leaf=100, 
                                max_bin=255,
                                n_estimators=500,
                                bagging_fraction=0.01,
                                min_sum_hessian_in_leaf=1,
                                importance_type='gain',
                                learning_rate=0.1, 
                                max_depth=-1, 
                                num_leaves=31,
                                verbose=-1)

In [19]:
model_lgbm = model_lgbm.fit(X_train, y_train)

In [20]:
print(classification_report(y_test, model_lgbm.predict(X_test),target_names=['class 1', 'class 2', 'class 3','class 4']))

              precision    recall  f1-score   support

     class 1       0.94      0.88      0.91       164
     class 2       0.89      0.92      0.90       311
     class 3       0.89      0.90      0.89       254
     class 4       0.97      0.97      0.97      1183

    accuracy                           0.95      1912
   macro avg       0.92      0.92      0.92      1912
weighted avg       0.95      0.95      0.95      1912



In [21]:
print("F1 score macro: ", f1_score(y_test, model_lgbm.predict(X_test), average = 'macro'))

F1 score macro:  0.9198888254141159


### Submit

In [22]:
useless_lgb_cols = X_train.columns[model_lgbm.feature_importances_ == 0].tolist()
model_lgbm = model_lgbm.fit(X_train.drop(columns = useless_lgb_cols), y_train)

In [ ]:
lgbm_submission = pd.DataFrame({'Id': data_test_id, 'Target': model_lgbm.predict(data_test.drop(columns = useless_lgb_cols))+1})

In [ ]:
lgbm_submission.to_csv('submission.csv', index = False)